In [1]:
from matplotlib import pyplot as plt
import IPython.display as ipd
import tensorflow as tf
from tensorflow import keras

2024-03-17 09:34:56.038755: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-17 09:34:56.062579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 09:34:56.406584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from dataset import get_datasets
from its_lru_v2 import ITS

In [4]:
#### hyper parameters that defines the structure of the model
num_classes = 31 # ds.get_labels()
sampled_frequencies = 129 # the number of frequency samples

learning_rate = 0.001
weight_decay = 0.005
batch_size = 64
num_epochs = 10000  # For real training, use num_epochs=100. 10 is a test value
# patch_size = 6  # Size of the patches to be extract from the input images
# num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 1
mlp_head_units = [
    526,
    256,
]  # Size of the dense layers of the final classifier


In [5]:
train, valid, test = get_datasets(batch_size=batch_size, type='mfccs')

/home/zuherj/miniconda3/envs/kws/lib/python3.9/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(
/home/zuherj/miniconda3/envs/kws/lib/python3.9/site-packages/tensorflow_datasets/core/dataset_builders/huggingface_dataset_builder.py:159: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  hf_names = hf_datasets.list_datasets()
2024-03-17 09:35:27.770468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-17 09:35:27.785171: I tensorflow/compiler/

In [6]:
# load the model weights
load_weights = False
if load_weights:
    ITS.load_weights(model_path)

In [12]:
import tensorflow as tf


class GatedMlpBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        inner_dim,
        outer_dim,
        non_linearity,
    ):
        super(GatedMlpBlock, self).__init__()
        self.inner_dense_non_linear = tf.keras.layers.Dense(
            units=inner_dim,
            activation=non_linearity,
        )
        self.inner_dense_linear = tf.keras.layers.Dense(
            units=inner_dim,
        )
        self.outer_dense = tf.keras.layers.Dense(
            units=outer_dim,
        )

    def call(self, input_seq):
        inner_non_linear = self.inner_dense_non_linear(input_seq)
        inner_linear = self.inner_dense_linear(input_seq)
        multiply = inner_non_linear * inner_linear
        return self.outer_dense(multiply)


class MultiQueryAttention(tf.keras.layers.Layer):
    def __init__(
        self,
        num_heads,
        proj_dim,
        dropout=0.0,
        kernel_regularizer=None,
    ):
        super(MultiQueryAttention, self).__init__()
        
        # define linear layers for key and value
        self.key_layer = tf.keras.layers.Dense(
            units=proj_dim,
            kernel_regularizer=kernel_regularizer,
        )
        self.value_layer = tf.keras.layers.Dense(
            units=proj_dim,
            kernel_regularizer=kernel_regularizer,
        )

        # define linear layers for query, as the number of heads
        self.query_layers = [tf.keras.layers.Dense(
            units=proj_dim,
            kernel_regularizer=kernel_regularizer,
        ) for _ in range(num_heads)]

        # define linear layer for output
        self.output_layer = tf.keras.layers.Dense(
            units=proj_dim,
            kernel_regularizer=kernel_regularizer,
        )


    def _compute_attn(
        self,
        query, # shape will be [B,S,d]
        input_keys, # shape will be [B,T,d]
        memory_keys, # shape will be [B,S,d]
        input_vals, # shape will be [B,T,d]
        memory_vals, # shape will be [B,S,d]
    ):
        # Assume S represents the number of memory cells and T represents the number of input cells
        # Compute the attention weights
        
        # Compute the score a memory cell gives to an input cell
        input_score = tf.matmul(query, input_keys, transpose_b=True)
        # Shape will be [B,S,T]. This will result in a matrix,
        # s.t. row i describes how much attention should the query i give all other input cells
        
        self_score = query * memory_keys
        self_score = tf.reduce_sum(self_score, axis=-1, keepdims=True)
        # Shape will be [B,S,1]. This will result in a vector,
        # s.t. element i describes how much attention should the query i give to itself

        # Concat self_score with input_score
        score = tf.concat([self_score, input_score], axis=-1)
        # Shape will be [B,S,T+1]. This will result in a matrix,
        # s.t. row i describes how much attention should the query i give to inputs and itself

        score /= tf.math.sqrt(tf.cast(tf.shape(input_keys)[-1], tf.float32))
        attn = tf.nn.softmax(score, axis=-1)

        # Break attn to [B,S,1] and [B,S,T]
        self_attn = attn[:, :, 0:1]
        input_attn = attn[:, :, 1:]

        value_of_input = tf.matmul(input_attn, input_vals) # shape will be [B,S,d]
        value_of_self = self_attn * memory_vals # shape will be [B,S,d]
        return value_of_input + value_of_self


    def call(self, input_seq, memory_cells):
        # query_seq is of shape (batch_size, input_size, key_dim)
        # store_seq is of shape (batch_size, store_seq, key_dim)
        # compute the attention weights
        ik = self.key_layer(input_seq)
        mk = self.key_layer(memory_cells)
        iv = self.value_layer(input_seq)
        mv = self.value_layer(memory_cells)
        attns = [self._compute_attn(q, ik, mk, iv, mv) for q in [layer(memory_cells) for layer in self.query_layers]]
        concat = tf.concat(attns, axis=-1)
        return self.output_layer(concat)
        

class StateTransformerBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        num_heads,
        projection_dim,
        inner_ff_dim,
        dropout=0.0,
        kernel_regularizer=None,
    ):
        super(StateTransformerBlock, self).__init__()
        # primitive properties
        self.num_heads = num_heads
        self.projection_dim = projection_dim
        
        # layers
        self.attention = MultiQueryAttention(
            num_heads=num_heads,
            proj_dim=projection_dim,
            dropout=dropout,
            kernel_regularizer=kernel_regularizer,
        )
        self.add1 = tf.keras.layers.Add()
        self.layernorm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.inner_dense = tf.keras.layers.Dense(
            units=inner_ff_dim,
            kernel_regularizer=kernel_regularizer,
            activation="relu",
        )
        self.outer_dense = GatedMlpBlock(
            inner_dim=inner_ff_dim,
            outer_dim=projection_dim,
            non_linearity="relu",
        )
        self.ff_dropout = tf.keras.layers.Dropout(dropout)
        self.add2 = tf.keras.layers.Add()
        self.layernorm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)


    def call(self, state_seq, input_seq):
        # state sequence is of shape (batch_size, num_of_state_cells, projection_dim)
        # input sequence is of shape (batch_size, input_size, projection_dim)
        # store_seq = tf.concat([state_seq, input_seq], axis=1)
        attention_output = self.attention(input_seq, state_seq)
        attention_output = self.add1([attention_output, state_seq])
        attention_output = self.layernorm_1(attention_output)
        inner_output = self.inner_dense(attention_output)
        outer_output = self.outer_dense(inner_output)
        outer_output = self.ff_dropout(outer_output)
        outer_output = self.add2([outer_output, attention_output])
        return self.layernorm_2(outer_output) # the output is of shape (batch_size, num_of_state_cells, projection_dim)
    

class ITSRU(tf.keras.layers.Layer):
    def __init__(
        self,
        num_heads,
        num_state_cells,
        projection_dim,
        inner_ff_dim,
        initial_state_trainability=False,
        dropout=0.0,
        kernel_regularizer=None,
    ):
        super(ITSRU, self).__init__()

        self.encoding = tf.keras.layers.Dense(
            units=projection_dim,
            kernel_regularizer=kernel_regularizer,
        )
        # Initialize the learnable initial state
        self.initial_state = self.add_weight(
            shape=(1, num_state_cells, projection_dim),
            initializer='random_normal',
            trainable=initial_state_trainability,
            name='initial_state'
        )
        # State TE layers
        self.calc_z = StateTransformerBlock(
            num_heads=num_heads,
            projection_dim=projection_dim,
            inner_ff_dim=inner_ff_dim,
            dropout=dropout,
            kernel_regularizer=kernel_regularizer,
        )
        self.calc_r = StateTransformerBlock(
            num_heads=num_heads,
            projection_dim=projection_dim,
            inner_ff_dim=inner_ff_dim,
            dropout=dropout,
            kernel_regularizer=kernel_regularizer,
        )
        self.calc_current_state = StateTransformerBlock(
            num_heads=num_heads,
            projection_dim=projection_dim,
            inner_ff_dim=inner_ff_dim,
            dropout=dropout,
            kernel_regularizer=kernel_regularizer,
        )


    def set_initial_state_trainability(self, trainable):
        self.initial_state._trainable = trainable


    def call(self, input_seq):
        # Assume that input is of size [B,T,S,D] where B is the batch size, T is the number of time steps, S is the sequence length at each timestep, and D is the feature dimension
        input_seq = self.encoding(input_seq)
        # initialize the state sequence
        batch_size = tf.shape(input_seq)[0]
        # Use the learnable initial state, replicate it for the whole batch
        state_t = tf.tile(self.initial_state, [batch_size, 1, 1])
        
        folds = tf.shape(input_seq)[1]
        states = tf.TensorArray(
            tf.float32,
            dynamic_size=True,
            size=0
        )
        for fold in range(folds):
            curr_input_seq = input_seq[:, fold, :, :]
            z = self.calc_z(state_t, curr_input_seq)
            r = self.calc_r(state_t, curr_input_seq)
            current_state = self.calc_current_state(r*state_t, curr_input_seq)
            state_t = (1 - z)*state_t + z*current_state
            states = states.write(fold, state_t)#.mark_used()
        
        return tf.transpose(
            states.stack(),
            [1, 0, 2, 3]
        )


class ITS(tf.keras.models.Model):
    def __init__(
        self,
        num_classes,
        num_heads,
        num_repeats,
        num_state_cells,
        input_seq_size,
        projection_dim,
        inner_ff_dim,
        initial_state_trainability=False,
        dropout=0.0,
        kernel_regularizer=None,
    ):
        super(ITS, self).__init__()
        # the input sequence size
        self.input_seq_size = input_seq_size
        
        # ITS recurrent units
        self.itsrus = [ ITSRU(
            num_heads=num_heads,
            num_state_cells=num_state_cells,
            projection_dim=projection_dim,
            inner_ff_dim=inner_ff_dim,
            initial_state_trainability=initial_state_trainability,
            dropout=dropout,
            kernel_regularizer=kernel_regularizer,
        ) for _ in range(num_repeats) ]
        
        self.label_token = self.add_weight(
            shape=(1, 1, projection_dim),
            initializer='random_normal',
            trainable=initial_state_trainability,
            name='initial_state'
        )
        self.mixer = StateTransformerBlock(
            num_heads=num_heads,
            projection_dim=projection_dim,
            inner_ff_dim=inner_ff_dim,
            dropout=dropout,
            kernel_regularizer=kernel_regularizer,
        )

        self.classifier = tf.keras.layers.Dense(
            units=num_classes,
            activation="softmax",
        )



    def call(self, input_seq):
        # input_seq is of shape (batch_size, input_size, feature_dim).
        # First of all, we will transform it to the shape (batch_size, folds, input_seq_size, projection_dim)
        # Pad the input sequence to the nearest multiple of input_seq_size
        input_seq_size = input_seq.shape[1]
        folds = tf.cast(tf.math.ceil(input_seq_size / self.input_seq_size), tf.int32)
        final_time_steps = folds * self.input_seq_size
        input_seq = tf.pad(
            input_seq,
            [[0, 0], [0, final_time_steps - input_seq_size], [0, 0]]
        )
        
        input_seq = tf.reshape(
            input_seq,
            [-1, folds, self.input_seq_size, input_seq.shape[-1]]
        )
        # pass the input sequence through the ITSRUs
        x = input_seq
        for itsru in self.itsrus:
            x = itsru(x)

        # mix the states of the last timestep with the label token
        # transform the label weight to the shape (batch_size, 1, projection_dim)
        # label_token = tf.tile(self.label_token, [tf.shape(x)[0], 1, 1])
        # x = self.mixer(label_token, x[:, -1, 0, :])
        # x = tf.squeeze(x, axis=1)

        return self.classifier(x[:, -1, 0, :])
        # return x

In [13]:
state_transformer = ITS(
    num_classes=31,
    num_repeats=2,
    num_heads=8,
    num_state_cells=10,
    input_seq_size=31,
    projection_dim=32,
    inner_ff_dim=64,
    dropout=0.1,
    kernel_regularizer=tf.keras.regularizers.l2(0.01),
)

In [14]:
state_transformer.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)


model_path = "./models/its_chkpnt/its_chkpnt.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

state_transformer_history = state_transformer.fit(
    train,
    validation_data=valid,
    epochs=21,
    # callbacks=[
    #     model_checkpoint_callback,
    # ],
)

Epoch 1/21
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorShape([None, 10, 32])
TensorSha

2024-03-17 09:45:55.673672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-17 09:45:55.760224: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa9cc00fcd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-17 09:45:55.760238: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-03-17 09:45:55.763221: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-17 09:45:55.857654: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-03-17 09:45:55.920146: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.

KeyboardInterrupt: 

In [ ]:
# Run both models TCResNet and StateTransformer for 30 epochs and graph the accuracy results
import matplotlib.pyplot as plt
results = {

}
for num_state_cells in [1, 4, 8, 12]:
    state_transformer = ITS(
        num_classes=31,
        num_repeats=2,
        num_heads=8,
        num_state_cells=num_state_cells,
        input_seq_size=31,
        projection_dim=32,
        inner_ff_dim=64,
        dropout=0.1,
        kernel_regularizer=tf.keras.regularizers.l2(0.01),
    )

    state_transformer.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )


    model_path = "./models/its_chkpnt/its_chkpnt.ckpt"
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        save_weights_only=True,
        save_freq="epoch",
        verbose=0,
    )

    state_transformer_history = state_transformer.fit(
        train,
        validation_data=valid,
        epochs=21,
        # callbacks=[
        #     model_checkpoint_callback,
        # ],
    )
    results[num_state_cells] = state_transformer_history.history['val_accuracy']

# write results to a csv file
import csv
with open('results.csv', 'w') as f:
    csv_writer = csv.writer(f)
    for key, values in results.items():
        csv_writer.writerow([key] + values)


In [13]:
import csv
with open('results.csv', 'r') as f:
    model_names = []
    data = []
    first_row = True
    csv_reader = csv.reader(f)
    for row in csv_reader:
        if first_row:
            model_names = row
            first_row = False
            continue
        else:
            data.append(row)
        

In [19]:
def get_file_lines():
    model_names = []
    data = []
    with open("results.csv", 'r') as f:
        reader = csv.reader(f)
        first_row = True
        for row in reader:
            if first_row:
                model_names = row
                first_row = False
            else:
                data.append(row)

    return model_names, data

In [22]:
model_names, data = get_file_lines()

In [23]:
print(model_names)
print(data)

['r=1,s=1', 'r=1,s=4', 'r=1,s=10', 'r=1,s=15', 'r=2,s=1', 'r=2,s=4', 'r=2,s=10', 'r=2,s=15', 'r=4,s=1', 'r=4,s=4', 'r=4,s=10', 'r=4,s=15']
[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'

In [26]:
repeats = 1
state_cells = 4
model_names.index(f"r={repeats},s={state_cells}")

1

In [10]:
from itertools import product

import csv

In [11]:
f = open('results.csv', 'w')
csv_writer = csv.writer(f)
csv_writer.writerow(list(product([1, 2, 3], [4, 5, 6])))

82

In [12]:
f.close()